<a href="https://colab.research.google.com/github/gadamsravya/NeuralStyleTransfer/blob/main/NeuralStyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

from tensorflow.python.keras.applications.vgg19 import VGG19

In [ ]:
#Defining the model to be used

model  = VGG19(include_top=False, weights='imagenet')
model.trainable = False
model.summary()

80142336/80134624 [==============================] - 3s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [ ]:
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.applications.vgg19 import preprocess_input
from tensorflow.python.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def load_and_preprocess_image(image_path):
  img = load_img(image_path)
  img = img_to_array(img)
  img = preprocess_input(img)
  img = np.expand_dims(img, axis = 0)
  return img

In [ ]:
def deprocess(x):
  x[:, :, 0] +=103.939
  x[:, :, 1] +=116.779
  x[:, :, 2] +=123.68

  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

def display_img(image):
  if(len(image.shape)==4):
    img = np.squeeze(image, axis = 0)
  img = deprocess(img)

  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img)

  return 


In [ ]:
display_img(load_and_preprocess_image("/content/style.jpg"))

In [ ]:
display_img(load_and_preprocess_image("/content/content.jpeg"))

In [ ]:
content_layer = 'block5_conv3'

style_layers = [
    'block2_conv2',
    'block3_conv3',
    'block4_conv4'
]

content_model = Model(inputs = model.input,
                      outputs = model.get_layer(content_layer).output
                      )

style_models = [Model(inputs = model.input, 
                     outputs = model.get_layer(layer).output) for layer in style_layers]

In [ ]:

def content_cost(content, generated):
  a_C = content_model(content)
  a_G = content_model(generated)
  cost = tf.reduce_mean(tf.square(a_C-a_G))

  return cost

In [ ]:
content = load_and_preprocess_image("/content/content1.png")
generated = load_and_preprocess_image("/content/content1.png")

cost = content_cost(content, generated)
print(cost)

tf.Tensor(0.0, shape=(), dtype=float32)


In [ ]:
def gramMatrix(a):
  n_C = int(a.shape[-1])
  a = tf.reshape(a, [-1, n_C])
  G = tf.matmul(a, a, transpose_a=True)/tf.cast(tf.shape(a)[0], tf.float32)
  return G

In [ ]:
def style_cost(style, generated):
  cost = 0
  for style_model in style_models:
    a_S = style_model(style)
    a_G = style_model(generated)
    
    G_S = gramMatrix(a_S)
    G_G = gramMatrix(a_G)

    costi = tf.reduce_mean(tf.square(G_S-G_G))
    cost = cost + costi

  return cost


In [ ]:
def training_function(content_path, style_path, iterations = 50, alpha = 10, beta = 80):
  content = load_and_preprocess_image(content_path)
  style = load_and_preprocess_image(style_path)

  generated = tf.Variable(content, dtype = tf.float32)

  optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 7.)

  best_cost = 1e12 + 1.0
  best_image = None
  for i in range(iterations):
    with tf.GradientTape() as g:
      print(i)
      J_C = content_cost(content, generated)
      
      J_S = style_cost(style, generated)
    
      J = alpha * J_C + beta * J_S
      print("Start")
      grads = g.gradient(J, generated)
      print("Next Start")
      optimizer.apply_gradients([(grads, generated)])
      print("Over")

      if best_cost>J:
        best_cost = J
        best_image = generated
    
  return best_image



In [ ]:
generated = training_function("/content/content1.jpg", "/content/style.jpeg")

In [ ]:
generated = generated.numpy()

In [ ]:
display_img(generated)

In [ ]:
style_img = load_and_preprocess_image("/content/366px-Van_Gogh_-_Terrasse_des_Cafés_an_der_Place_du_Forum_in_Arles_am_Abend1.jpeg")
style_layers1 = [
                 'block1_conv1',
                 'block1_conv2',
                 'block2_conv1',
                 'block2_conv2',
                 'block3_conv1',
                 'block3_conv2',
                 'block3_conv3',
                 'block3_conv4',
                 'block4_conv1',
                 'block4_conv2',
                 'block4_conv3',
                 'block4_conv4',
                 'block5_conv1',
                 'block5_conv2',
                 'block5_conv3',
                 'block5_conv4',
]
style_models1 = [Model( inputs = model.input, ouputs = model.get_layer(name = style_layer)) for style_layer in style_layers1]
for style_model in style_models1:
  style_img = style_model(style1)
  display_img(style_img)